In [22]:
import pandas as pd
import networkx as nx

## Load dữ liệu

In [23]:
nodes_df = pd.read_csv("/kaggle/input/network/nodes.csv")
edges_df = pd.read_csv("/kaggle/input/network/edges.csv")

print("Số lượng node:", len(nodes_df))
print("Số lượng cạnh:", len(edges_df))

nodes_df.head(), edges_df.head()


Số lượng node: 3939
Số lượng cạnh: 10122


(                                                link  \
 0       /wiki/Gi%E1%BA%A3i_Nobel_V%E1%BA%ADt_l%C3%BD   
 1      /wiki/Gi%E1%BA%A3i_Nobel_H%C3%B3a_h%E1%BB%8Dc   
 2  /wiki/Gi%E1%BA%A3i_Nobel_Sinh_l%C3%BD_h%E1%BB%...   
 3      /wiki/Gi%E1%BA%A3i_Nobel_V%C4%83n_h%E1%BB%8Dc   
 4        /wiki/Gi%E1%BA%A3i_Nobel_H%C3%B2a_b%C3%ACnh   
 
                                 name   type  
 0                  Giải Nobel Vật lý  EVENT  
 1                 Giải Nobel hóa học  EVENT  
 2  Giải Nobel Sinh lý học hoặc Y học  EVENT  
 3                 Giải Nobel Văn học  EVENT  
 4                Giải Nobel Hòa bình  EVENT  ,
                             src  \
 0          /wiki/George_Stigler   
 1  /wiki/Martinus_J._G._Veltman   
 2       /wiki/Tjalling_Koopmans   
 3          /wiki/Robert_Mundell   
 4       /wiki/Christian_de_Duve   
 
                                                  des  \
 0           /wiki/%C4%90%E1%BA%A1i_h%E1%BB%8Dc_Brown   
 1  /w/index.php?title=Peter_van_Nieuwenh

In [24]:
# Làm sạch dữ liệu
nodes_df = nodes_df.dropna(subset=['link', 'name'])
nodes_df['link'] = nodes_df['link'].astype(str).str.strip()
nodes_df['name'] = nodes_df['name'].astype(str).str.strip()

edges_df['src'] = edges_df['src'].astype(str).str.strip()
edges_df['des'] = edges_df['des'].astype(str).str.strip()
edges_df['type'] = edges_df['type'].astype(str).str.strip()

# Mapping giữa link và tên
link_to_name = dict(zip(nodes_df['link'], nodes_df['name']))
name_to_link = {v: k for k, v in link_to_name.items()}

In [25]:
WEIGHT_MAP = {
    "Con cái": 1,
    "Phối ngẫu": 1,
    "Giải thưởng nổi bật": 1,
    "Người hướng dẫn luận án tiến sĩ": 2,
    "Các nghiên cứu sinh nổi tiếng": 2,
    "Ảnh hưởng tới": 3,
    "Nơi công tác": 4,
    "Trường lớp": 4,
    "Sinh": 5,
    "Mất": 5,
}

In [26]:
def get_weight(rel_type):
    return WEIGHT_MAP.get(rel_type.strip(), 10) 

In [27]:
G = nx.DiGraph()

for _, row in edges_df.iterrows():
    src = row['src']
    des = row['des']
    rel = row['type']
    
    if src in link_to_name and des in link_to_name:
        w = get_weight(rel)
        G.add_edge(src, des, weight=w, rel=rel)
        G.add_edge(des, src, weight=w+2, rel=rel + " (ngược)")

print(f"   • Số node: {G.number_of_nodes():,}")
print(f"   • Số cạnh: {G.number_of_edges():,}\n")

   • Số node: 3,717
   • Số cạnh: 15,554



In [28]:
def tim_duong_di_trong_so(tu, den):
    tu = tu.strip()
    den = den.strip()
    
    if tu not in name_to_link:
        print(f"Không tìm thấy node: {tu}")
        return
    if den not in name_to_link:
        print(f"Không tìm thấy node: {den}")
        return
    
    src = name_to_link[tu]
    dst = name_to_link[den]
    
    try:
        path = nx.dijkstra_path(G, source=src, target=dst, weight='weight')
        total_weight = nx.dijkstra_path_length(G, source=src, target=dst, weight='weight')
        
        print("="*50)
        print(f"Đường đi ngắn nhất từ {tu} → {den}")
        print(f"Tổng trọng số = {total_weight}, Số bước = {len(path)-1}")
        print("-"*50)
        
        for i in range(len(path)-1):
            u = path[i]
            v = path[i+1]
            edge_data = G[u][v]
            print(f"Bước {i+1}: {link_to_name[u]}")
            print(f"        Trọng số = {edge_data['weight']}, Quan hệ = {edge_data['rel']}")
            print(f"        → {link_to_name[v]}\n")
        
        print("="*50 + "\n")
        
    except nx.NetworkXNoPath:
        print(f"Không có đường đi từ {tu} đến {den}\n")
    except Exception as e:
        print("Lỗi:", e)


In [ ]:
if __name__ == "__main__":
    while True:
        start = input("Nhập node bắt đầu (hoặc 'exit' để thoát): ").strip()
        if start.lower() in ["exit", "e"]:
            break
        end = input("Nhập node kết thúc: ").strip()
        tim_duong_di_trong_so(start, end)

Nhập node bắt đầu (hoặc 'exit' để thoát):  Albert Einstein
Nhập node kết thúc:  Max Planck


Đường đi ngắn nhất từ Albert Einstein → Max Planck
Tổng trọng số = 16, Số bước = 4
--------------------------------------------------
Bước 1: Albert Einstein
        Trọng số = 2, Quan hệ = Người hướng dẫn luận án tiến sĩ
        → Alfred Kleiner

Bước 2: Alfred Kleiner
        Trọng số = 4, Quan hệ = Nơi công tác
        → Đại học Zürich

Bước 3: Đại học Zürich
        Trọng số = 6, Quan hệ = Nơi công tác (ngược)
        → Max von Laue

Bước 4: Max von Laue
        Trọng số = 4, Quan hệ = Các nghiên cứu sinh nổi tiếng (ngược)
        → Max Planck




Nhập node bắt đầu (hoặc 'exit' để thoát):  Max Planck
Nhập node kết thúc:  Niels Bohr


Đường đi ngắn nhất từ Max Planck → Niels Bohr
Tổng trọng số = 15, Số bước = 3
--------------------------------------------------
Bước 1: Max Planck
        Trọng số = 4, Quan hệ = Nơi công tác
        → Đại học Göttingen

Bước 2: Đại học Göttingen
        Trọng số = 6, Quan hệ = Nơi công tác (ngược)
        → Wolfgang Ernst Pauli

Bước 3: Wolfgang Ernst Pauli
        Trọng số = 5, Quan hệ = Ảnh hưởng tới (ngược)
        → Niels Bohr




Nhập node bắt đầu (hoặc 'exit' để thoát):  N2
Nhập node kết thúc:  B2


Không tìm thấy node: N2
